<a href="https://colab.research.google.com/github/DiMorten/FCN_ConvLSTM_Crop_Recognition_Open_Set/blob/coords5/train_and_evaluate_openset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Dependencies

In [1]:
!pip install icecream
#%tensorflow_version 1.x
import os
!pip install kora
from kora import drive
import time
!pip install colorama

dataset = 'lm'
ds_path='/content/drive/My Drive/PhD/datasets/' + dataset + '_data/'

     |████████████████████████████████| 57 kB 2.9 MB/s 
     |████████████████████████████████| 56 kB 5.0 MB/s 


In [ ]:

from google.colab import drive
drive.mount('/content/drive')

In [ ]:
git_clone = True

if git_clone == True:
  os.chdir('/content')
  %rm -rf FCN_ConvLSTM_Crop_Recognition_Open_Set
  !git clone --branch coords5 https://github.com/DiMorten/FCN_ConvLSTM_Crop_Recognition_Open_Set.git

## Download images into proper folder

In [ ]:
from distutils.dir_util import copy_tree
copy_tree('/content/drive/MyDrive/PhD/datasets/'+dataset+'_data', "/content/FCN_ConvLSTM_Crop_Recognition_Open_Set/dataset/dataset/"+dataset+"_data")

#!cp -r '/content/drive/MyDrive/PhD/datasets/'+dataset+'_data' /content/FCN_ConvLSTM_Crop_Recognition_Open_Set/dataset/dataset/

In [ ]:
os.chdir('/content/FCN_ConvLSTM_Crop_Recognition_Open_Set/networks/convlstm_networks/train_src')
os.getcwd()
os.listdir()

In [ ]:
from colorama import init
init()
from tensorflow.keras.layers import Input, Dense, Conv2D, MaxPool2D, Flatten, Dropout, Conv2DTranspose
# from tensorflow.keras.callbacks import ModelCheckpoint , EarlyStopping
from tensorflow.keras.optimizers import Adam,Adagrad 
from tensorflow.keras.models import Model
from tensorflow.keras import backend as K
import tensorflow.keras as keras

import numpy as np
from sklearn.utils import shuffle
import cv2
import argparse
import tensorflow as tf

from tensorflow.keras.models import *
from tensorflow.keras.layers import *
from tensorflow.keras.optimizers import *
from tensorflow.keras import metrics
import sys
import glob

from sklearn.metrics import confusion_matrix,f1_score,accuracy_score,classification_report
# Local
from densnet import DenseNetFCN
from densnet_timedistributed import DenseNetFCNTimeDistributed

#from metrics import fmeasure,categorical_accuracy
import deb
from keras_weighted_categorical_crossentropy import weighted_categorical_crossentropy, sparse_accuracy_ignoring_last_label, weighted_categorical_crossentropy_ignoring_last_label, categorical_focal_ignoring_last_label, weighted_categorical_focal_ignoring_last_label
from tensorflow.keras.models import load_model
from tensorflow.keras.layers import ConvLSTM2D, UpSampling2D, multiply
from tensorflow.keras.regularizers import l1,l2
import time
import pickle
#from tensorflow.keras_self_attention import SeqSelfAttention
import pdb
import pathlib
from pathlib import Path, PureWindowsPath
from tensorflow.keras.layers import Conv3DTranspose, Conv3D

from tensorflow.keras.callbacks import EarlyStopping
import tensorflow as tf
from collections import Counter


#from datagenerator import DataGenerator
from generator import DataGenerator, DataGeneratorWithCoords, DataGeneratorWithCoordsRandom

import matplotlib.pyplot as plt
sys.path.append('../../../dataset/dataset/patches_extract_script/')
from dataSource import DataSource, SARSource, OpticalSource, Dataset, LEM, LEM2, CampoVerde, OpticalSourceWithClouds, Humidity
from model_input_mode import MIMFixed, MIMVarLabel, MIMVarSeqLabel, MIMVarLabel_PaddedSeq, MIMFixedLabelAllLabels, MIMFixed_PaddedSeq
from parameters.params_train import ParamsTrain

from icecream import ic
from monitor import Monitor, MonitorNPY, MonitorGenerator, MonitorNPYAndGenerator
import natsort
from model import NetModel, ModelFit, ModelLoadGeneratorWithCoords
from dataset import Dataset, DatasetWithCoords

from patch_extractor import PatchExtractor
ic.configureOutput(includeContext=False)
np.random.seed(2021)
#tf.random.set_seed(2021)

from main import TrainTest

In [ ]:
!nvidia-smi

## Set parameters

Parameters can be modified in /content/FCN_ConvLSTM_Crop_Recognition_Open_Set/networks/convlstm_networks/train_src/parameters/params_train.py

In [ ]:
from pathlib import Path

openSetMethod = 'OpenPCS'

paramsTrain = ParamsTrain('parameters/', openSetMethod = openSetMethod)
paramsTrain.mim = MIMFixed_PaddedSeq()

paramsTrain.getFullIms = True
paramsTrain.coordsExtract = True
paramsTrain.train = True

paramsTrain.train_overlap_percentage = 0
paramsTrain.trainGeneratorRandom = False
paramsTrain.patch_len = 32
paramsTrain.stride = int(paramsTrain.patch_len - paramsTrain.patch_len * paramsTrain.train_overlap_percentage)
paramsTrain.patch_step_train = paramsTrain.stride
paramsTrain.patch_step_test = paramsTrain.patch_len # to do: paramsTrain.getCalculatedParams() does these calculations

paramsTrain.dataset = dataset
paramsTrain.seq_date = 'mar'
paramsTrain.path = Path("../../../dataset/dataset/") / (paramsTrain.dataset + "_data")

paramsTrain.test_overlap_percentage = 0

paramsTrain.dataSource = SARSource()
#paramsTrain.openSetMethod = None
paramsTrain.selectMainClasses = True




trainTest = TrainTest(paramsTrain)

patchExtractor = PatchExtractor(paramsTrain, trainTest.ds)	

## Download or load sequence of images


In [ ]:
os.getcwd()

In [ ]:
#os.chdir('../../../dataset/dataset/lm_data/in_sar/')
#os.getcwd()
#os.listdir()

In [ ]:
if paramsTrain.getFullIms == True:
  patchExtractor.getFullIms()	
else:
  patchExtractor.fullImsLoad()


## Extract coords of image patches

In [ ]:

if paramsTrain.coordsExtract == True:
  patchExtractor.extract()

del patchExtractor



## Train


In [ ]:
	trainTest.setData()


	trainTest.preprocess(paramsTrain.model_name_id) # move into if

	trainTest.setModel()

	if paramsTrain.train == True:
		trainTest.train()
	else:
    # copy pre trained model to paramsTrain.model_name_id
		trainTest.modelLoad(paramsTrain.model_name_id)

## Evaluate


In [ ]:
from parameters.params_mosaic import ParamsReconstruct
import pathlib

pathlib.Path('results/spatial_results').mkdir(parents=True, exist_ok=True)

paramsMosaic = ParamsReconstruct(paramsTrain)

paramsMosaic.mosaic_flag = True
paramsMosaic.open_set_mode = False

trainTest.evaluate(paramsMosaic)
